In [1]:
import tensorflow as tf

2024-02-16 00:44:33.833560: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-16 00:44:33.923592: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 00:44:34.326802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 00:44:34.326874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 00:44:34.409585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import os
import cv2
import keras_ocr
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
PATH = "./data/img/01245.png"
# files = os.listdir(PATH)

In [4]:
def compute_grad(image_gray, mode:str)->np.ndarray:
    """
    Function to compute the gradients magnitude of the image 
    set mode to "double" if you want to apply it two times
    """
    # Compute the gradients using the Sobel operator
    dx = cv2.Sobel(image_gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(image_gray, cv2.CV_64F, 0, 1, ksize=3)

    # Compute the magnitude and direction of the gradients
    mag = np.sqrt(dx**2 + dy**2)
    
    if mode == "double":
        # Compute the gradients using the Sobel operator
        dx = cv2.Sobel(mag, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(mag, cv2.CV_64F, 0, 1, ksize=3)

        # Compute the magnitude and direction of the gradients
        magmag = np.sqrt(dx**2 + dy**2)
        return (magmag/magmag.max()*255).astype(np.uint8)
        
    return (mag/mag.max()*255).astype(np.uint8)




def detect_draw(pipeline, image_gray, viz):
    if len(image_gray.shape) == 3:  # Check if image is not grayscale
        image_gray = cv2.cvtColor(image_gray, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2RGB)
    # img=image_gray
    #read image from the an image path (a jpg/png file or an image url)
    # Prediction_groups is a list of (word, box) tuples
    b = pipeline.recognize([img])
    #print image with annotation and boxes
    # if viz:
    #     keras_ocr.tools.drawAnnotations(image=img, predictions=b[0])
    
    
    x_min = int(min([b[0][i][1][:,0].min() for i in range(len(b[0]))]))
    x_max = int(max([b[0][i][1][:,0].max() for i in range(len(b[0]))]))

    y_min = int(min([b[0][i][1][:,1].min() for i in range(len(b[0]))]))
    y_max = int(max([b[0][i][1][:,1].max() for i in range(len(b[0]))]))
    return (x_min,y_min,x_max,y_max)

def remove(image_gray, bb, offset):
    # Create a mask of the same size as the image
    mask = np.ones_like(image_gray)*255

    # Draw a white rectangle on the mask within the bounding box
    cv2.rectangle(mask, (bb[0]-offset, bb[1]-offset), (bb[2]+offset, bb[3]+offset), (0,0,0), -1)

    # Apply the mask to the original image
    masked_image = cv2.bitwise_and(image_gray, mask)
    
    return masked_image

In [5]:
def main(pipeline, image_gray, offset):
    mag = compute_grad(image_gray,"single")
    bb = detect_draw(pipeline, mag, viz=True)
    masked_image = remove(image_gray, bb, offset)
    return masked_image

In [6]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /home/annamalaisenthil112/.keras-ocr/craft_mlt_25k.h5
Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.
Looking for /home/annamalaisenthil112/.keras-ocr/crnn_kurapan.h5


In [7]:
image_gray  = np.array(Image.open(PATH))
masked_image = main(pipeline,image_gray,40)
# fig, ax = plt.subplots(1,2,figsize=(20,7))

# ax[0].imshow(image_gray)
# ax[1].imshow(masked_image)
# masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
# if len(masked_image.shape) == 3:  # Check if image is not grayscale
#     masked_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2GRAY)
masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
cv2.imwrite("cat-text.png", masked_image)

1/1 [==============================] - 3s 3s/step


True

In [8]:
masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
cv2.imwrite("cat-text.png", masked_image)

error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<1>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 3
